In [ ]:
import ...

## load model

In [ ]:
model = ...

## Top 10 attributes class 1

In [ ]:
# extract the top 10 attributes leading to one class

## Top 10 attributes class 2

In [ ]:
# extract the top 10 attributes leading to the other class

## lime

In [ ]:
# use lime to explain the prediction result of a test string

In [ ]:
# use the built-in lime visualizations

## eli5

In [ ]:
# use eli5 to show the top N features contributing to one class

In [ ]:
# use eli5 to visualize the most important features for a sample text

## send your candidate messages to the server for evaluation

In [ ]:
...